#### Object: Train an LSTM to mimic Russell’s style and thoughts

In [14]:
import numpy as np
!pip install keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import string

1. Download 4 books written by Bertrand Russell

2. Import these text files in and reformat the data

In [60]:
f1 = open('../data/1.txt','r', encoding="ISO-8859-1").read()
f1 = f1.lower()
f1 = f1.replace('\n',' ')

In [61]:
f2 = open('../data/2.txt','r', encoding="ISO-8859-1").read()
f2 = f2.lower()
f2 = f2.replace('\n',' ')
f3 = open('../data/3.txt','r', encoding="ISO-8859-1").read()
f3 = f3.lower()
f3 = f3.replace('\n',' ')
f4 = open('../data/4.txt','r', encoding="ISO-8859-1").read()
f4 = f4.lower()
f4 = f4.replace('\n',' ')

# CONCANTENATE 4 FILES TOGEETHER 
files = f1+f2+f3+f4
files = files[1:]

In [62]:
# remove punctuations in the concatenated text
files = files.translate(str.maketrans('', '', string.punctuation))

3. Encode characters in ASCII order

In [63]:
sorted_chars = sorted(list(set(files)))
char_ind = dict()
for ind, char in enumerate(sorted_chars):
    char_ind[char] = ind

In [64]:
# report the number of all characters used
chars_used = len(char_ind)
print(f'There are totally {chars_used} characters used in the text.')

There are totally 73 characters used in the text.


In [65]:
total_chars = len(files)
print(f'The file contains {total_chars} characters in total.')

The file contains 1548600 characters in total.


4. Rescale characters and set up input, output sets

In [66]:
win = 100-1
input_sets = []
output_sets = []
for i in range(0, total_chars - win):
    try:
        eles_in = files[i:i + win]
        ele_out = files[i + win]
    except:
        continue
    input_lst = []
    for c in eles_in:
        input_lst.append(char_ind[c])
    input_sets.append(input_lst)
    output_sets.append(char_ind[ele_out])
# total number of input sets
num_input_sets = len(input_sets)
data_X = np.reshape(input_sets, (num_input_sets, win, 1))


In [67]:
# rescaled input numbers
data_X = data_X / float(chars_used)

In [68]:
# reformat the output with the one-hot encoding scheme
data_y = np_utils.to_categorical(output_sets)

5. Build a single hidden layer for the LSTM with N = 256 (or less) memory units; use Softmax output layer

In [69]:
lstm = Sequential()
lstm.add(LSTM(256, input_shape=(data_X.shape[1], data_X.shape[2])))
lstm.add(Dropout(0.2))
lstm.add(Dense(data_y.shape[1], activation='softmax'))
lstm.compile(loss='categorical_crossentropy', optimizer='adam')

In [70]:
lstm.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 256)               264192    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 73)                18761     
Total params: 282,953
Trainable params: 282,953
Non-trainable params: 0
_________________________________________________________________


6. Choose 16 epochs for training; use model checkpointing to keep the network weights

In [71]:
weights_file="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(weights_file, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_lst = [checkpoint]

In [72]:
lstm.fit(data_X, data_y, epochs=16, batch_size=128, callbacks=callbacks_lst)

Epoch 1/16
12098/12098 [==============================] - 2780s 230ms/step - loss: 2.5590

Epoch 00001: loss improved from inf to 2.55896, saving model to weights-improvement-01-2.5590.hdf5
Epoch 2/16
12098/12098 [==============================] - 4739s 392ms/step - loss: 2.2389

Epoch 00002: loss improved from 2.55896 to 2.23888, saving model to weights-improvement-02-2.2389.hdf5
Epoch 3/16
12098/12098 [==============================] - 2607s 215ms/step - loss: 2.0508

Epoch 00003: loss improved from 2.23888 to 2.05078, saving model to weights-improvement-03-2.0508.hdf5
Epoch 4/16
12098/12098 [==============================] - 2595s 215ms/step - loss: 1.9453

Epoch 00004: loss improved from 2.05078 to 1.94533, saving model to weights-improvement-04-1.9453.hdf5
Epoch 5/16
12098/12098 [==============================] - 2594s 214ms/step - loss: 1.8770

Epoch 00005: loss improved from 1.94533 to 1.87700, saving model to weights-improvement-05-1.8770.hdf5
Epoch 6/16
12098/12098 [==========

7. Generate statement with lstm model with weights what give out the least loss

In [50]:
ind_chars = dict()
for ind, char in enumerate(sorted_chars):
    ind_chars[ind] = char

initializing_sentence = '''There are those who take mental phenomena naively, just as they
would physical phenomena. This school of psychologists tends not to
emphasize the object.'''
initializing_sentence = initializing_sentence.translate(str.maketrans('', '', string.punctuation))
initializing_sentence = initializing_sentence.replace('\n',' ')

In [54]:
# after reformat:
initializing_sentence = initializing_sentence.lower()
initializing_sentence

'there are those who take mental phenomena naively just as they would physical phenomena this school of psychologists tends not to emphasize the object'

In [73]:
ind_sentence = []
for char in initializing_sentence:
    ind_sentence.append(char_ind[char])

In [74]:
# report th number of characters in this sentence 
num_ini_sen = len(ind_sentence)
print(f'There are {num_ini_sen} characters in the initializing sentence.')

There are 150 characters in the initializing sentence.


8. Let's write out the result:

In [75]:
import sys

In [79]:
pat = ind_sentence[0:99]
for i in range(1000):
    x_val = np.reshape(pat, (1, len(pat), 1))
    x_val = x_val / float(chars_used)
    pred = lstm.predict(x_val, verbose=0)
    index = np.argmax(pred)
    result = ind_chars[index]
    sys.stdout.write(result)
    pat.append(index)
    pat = pat[1:len(pat)]
    

 and the sensedata the sensedata the sensedata the sensedata and the sensedata the sensedata and the sensedata the sensedata and the sensedata the sensedata and the sensedata the sensedata and the sensedata the sensedata and the sensedata the sensedata and the sensedata the sensedata and the sensedata the sensedata and the sensedata the sensedata and the sensedata the sensedata and the sensedata the sensedata and the sensedata the sensedata and the sensedata the sensedata and the sensedata the sensedata and the sensedata the sensedata and the sensedata the sensedata and the sensedata the sensedata and the sensedata the sensedata and the sensedata the sensedata and the sensedata the sensedata and the sensedata the sensedata and the sensedata the sensedata and the sensedata the sensedata and the sensedata the sensedata and the sensedata the sensedata and the sensedata the sensedata and the sensedata the sensedata and the sensedata the sensedata and the sensedata the sensedata and the sen